In [ ]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName('sql_window').getOrCreate() 

df = spark.read.csv('/workspaces/machine_learning-projects-/pending_work/pyspark task/small_media_data.csv', header = True,
                    inferSchema = True) 


'''
ROW_NUMBER, RANK, LEAD, LAG
'''

# sql 

df.createOrReplaceTempView('media_table') 

spark.sql(
    '''
        with temp_temp as (
            select 
                row_number() over( partition by media_type order by likes desc ) as ranking, 
                rank() over(partition by media_type order by likes desc) as rank_ranking , 
                lead(likes) over ( order by timestamp ) as lead_function , 
                lag(likes) over( order by timestamp) as leg_function
            from media_table 
        )
        
        select * from temp_temp ; 
    
    '''
).show() 


25/03/26 04:02:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/26 04:02:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/26 04:02:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/26 04:02:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+------------+-------------+------------+
|ranking|rank_ranking|lead_function|leg_function|
+-------+------------+-------------+------------+
|      1|           1|          469|         994|
|      2|           2|           20|         462|
|      3|           3|          617|         819|
|      4|           4|          899|         263|
|      5|           5|          231|         643|
|      6|           6|          259|         131|
|      7|           7|          784|         521|
|      8|           8|           69|         999|
|      9|           9|          792|         231|
|     10|          10|          380|         247|
|     11|          11|          496|         577|
|     12|          12|           83|         955|
|     13|          13|          435|         253|
|     14|          14|          969|         287|
|     15|          15|          608|         111|
|     16|          16|          702|         816|
|     17|          17|           72|         426|


In [6]:
from pyspark.sql.window import Window 
from pyspark.sql.functions import  row_number 

windows = Window.partitionBy(df.user_id).orderBy(df.likes.desc()) 
df = df.withColumn('ranking', row_number().over(windows) ) 




In [12]:
from pyspark.sql.functions import rank , col 
from pyspark.sql.window import Window 


windows = Window.partitionBy(col('user_id')).orderBy(col('likes').desc())  
df = df.withColumn('rank_ranking', rank().over(windows)) 
df.show()

+-------+-------+--------------------+-----+------+--------+----------+-------+------------+
|post_id|user_id|           timestamp|likes|shares|comments|media_type|ranking|rank_ranking|
+-------+-------+--------------------+-----+------+--------+----------+-------+------------+
|    922|      1|2025-03-08 16:57:...|  999|   125|     192|     video|      1|           1|
|     94|      1|2025-03-19 15:33:...|  961|    69|      71|      text|      2|           2|
|    927|      1|2025-03-12 21:42:...|  941|   495|      24|      text|      3|           3|
|     17|      1|2025-03-11 23:20:...|  813|   258|     182|     image|      4|           4|
|    655|      1|2025-02-25 20:43:...|  714|   403|      78|     image|      5|           5|
|     78|      1|2025-03-07 18:27:...|  604|   341|     135|      text|      6|           6|
|      8|      1|2025-03-14 06:45:...|  537|   358|      45|     video|      7|           7|
|    954|      1|2025-03-05 21:33:...|  158|    20|     193|      text

In [19]:
from pyspark.sql.functions import  lead , lag, col 
from pyspark.sql.window import Window 

windows = Window.orderBy(col('likes').desc()) 

df = df.withColumn('lead', lead(col('likes')).over(windows)) 
df = df.withColumn('leg', lag(col('likes')).over(windows)) 

df.show()

+-------+-------+--------------------+-----+------+--------+----------+-------+------------+----+----+
|post_id|user_id|           timestamp|likes|shares|comments|media_type|ranking|rank_ranking|lead| leg|
+-------+-------+--------------------+-----+------+--------+----------+-------+------------+----+----+
|    922|      1|2025-03-08 16:57:...|  999|   125|     192|     video|      1|           1| 998|NULL|
|     94|      1|2025-03-19 15:33:...|  961|    69|      71|      text|      2|           2| 961| 961|
|    927|      1|2025-03-12 21:42:...|  941|   495|      24|      text|      3|           3| 940| 941|
|     17|      1|2025-03-11 23:20:...|  813|   258|     182|     image|      4|           4| 813| 814|
|    655|      1|2025-02-25 20:43:...|  714|   403|      78|     image|      5|           5| 711| 714|
|     78|      1|2025-03-07 18:27:...|  604|   341|     135|      text|      6|           6| 601| 606|
|      8|      1|2025-03-14 06:45:...|  537|   358|      45|     video|  

25/03/26 04:00:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/26 04:00:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/26 04:00:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/26 04:00:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
